In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Context

In [0]:
var log = await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
var log = await Import.FromFile("../Files/TransactionalData/Actuals_CH_2020_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
var defaultVarsBE = Workspace.Query<IfrsVariable>().ToArray();

# Scenario Cash flows

In [0]:
var cashflows_scenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var cashflows_scenarioTest_equalTo_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";

In [0]:
var log = await Import.FromString(cashflows_scenarioTest).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "Test", Year = 2020, Month = 12});
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "Test", Year = 2020, Month = 12});
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().Be(0);
diffs.Where(x => !(new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

# Scenario Actuals

In [0]:
var actuals_scenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-308";

In [0]:
var actuals_scenarioTest_equalTo_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-280";

In [0]:
var log = await Import.FromString(actuals_scenarioTest).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "Test", Year = 2020, Month = 12});
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new{ReportingNode = "CH", Scenario = "Test", Year = 2020, Month = 12});
var diffs = Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance());
diffs.Where(x => (new[]{"A","AA","OA","DA"}).Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

# Robustness: back to Best Estimate

In [0]:
var log = await Import.FromString(actuals_scenarioTest_equalTo_BestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
var log = await Import.FromString(cashflows_scenarioTest_equalTo_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log

In [0]:
Workspace.Query<IfrsVariable>().Except(defaultVarsBE, IfrsVariableComparer.Instance()).ToArray().Length.Should().Be(0);